Cleaning the data

In [15]:
import pandas as pd

In [16]:
df = pd.read_csv('data/hayday_item_data.csv')
df.head()

,Name,Level,Max. price,Time,XP,Needs,Source,Per boat crate
0,Wheat,1,3,2 min,1,Wheat (1),Field (1st crop),50-101
1,Egg,1,18,20 min,2,Chicken feed (1),Chicken (1st animal product),15-24
2,Corn,2,7,5 min,1,Corn (1),Field (2nd crop),50-87
3,Bread,2,21,5 min★★★ 4 min,3,Wheat (3),Bakery,14-26
4,Chicken feed,3,7,5 min★★★ 4 min,1,Corn (1)\nWheat (2),Feed Mill,NaN


Cleaning columns

In [17]:
df = df.drop(columns=['XP', 'Needs', 'Per boat crate'])

In [18]:
df['Level'] = df['Level'].apply(lambda x: int(x.split()[0]))
df['Max. price'] = df['Max. price'].apply(lambda x: int(x))

In [19]:
df.head()

,Name,Level,Max. price,Time,Source
0,Wheat,1,3,2 min,Field (1st crop)
1,Egg,1,18,20 min,Chicken (1st animal product)
2,Corn,2,7,5 min,Field (2nd crop)
3,Bread,2,21,5 min★★★ 4 min,Bakery
4,Chicken feed,3,7,5 min★★★ 4 min,Feed Mill


Cleaning time column

In [20]:
df.loc[df['Time'].str.contains('Instant') , 'Time'] = 0

In [21]:
import re

def clean_time_string(time_txt: str):

    if(type(time_txt) == int):
        return time_txt
    
    allowed_characters = '1234567890minhd '
    split_poz = -1

    for i in range(len(time_txt)):
        if time_txt[i] not in allowed_characters:
            if split_poz == -1:
                split_poz = i
    
    time_txt = re.sub(r'[^a-zA-Z0-9 ]+', '', time_txt)

    if split_poz != -1:
        time_txt = time_txt[:split_poz]

    time_txt = time_txt.strip()

    time_list = time_txt.split()
    time_int = 0

    for i in range(len(time_list)-1):
        try:
            number = int(time_list[i])

            metric = time_list[i+1]
            if metric == 'h':
                number = number * 60
            elif metric == 'd':
                number = number * 60 * 24
            
            time_int += number
        except:
            pass

    
    return time_int

print(clean_time_string('2 h 10 min 10 d★★★ 1 h 42 min'))
print(clean_time_string('2 h 10 min'))

14530
130


In [22]:
df['Time'] = df['Time'].apply(lambda x: clean_time_string(x))

In [23]:
df.rename(columns={'Time': 'Time (min)', 'Max. price' : 'Price'}, inplace=True)

In [24]:
df.head()

,Name,Level,Price,Time (min),Source
0,Wheat,1,3,2,Field (1st crop)
1,Egg,1,18,20,Chicken (1st animal product)
2,Corn,2,7,5,Field (2nd crop)
3,Bread,2,21,5,Bakery
4,Chicken feed,3,7,5,Feed Mill


In [25]:
df.to_csv('data/clean_hayday_item_data.csv', index=False)

print("Data saved to data/clean_hayday_item_data.csv")

Data saved to data/clean_hayday_item_data.csv
